#Optimal Hangman Strategy Using Statistics & Probability

##Import Relevant Libraries

In [ ]:
# For Hangman
from collections      import defaultdict
from urllib.request   import urlopen
from random           import shuffle
from random           import choice

# For exporting as CSV
import numpy as np

##Download Dictionary Datasets

In [ ]:
# Remove dataset files if they were previously downloaded
!rm words.txt
!rm connectives.txt
!rm propernames.txt

In [ ]:
# Get datasets from URLs
!wget https://svnweb.freebsd.org/csrg/share/dict/words?view=co
!wget https://svnweb.freebsd.org/csrg/share/dict/connectives?view=co
!wget https://svnweb.freebsd.org/csrg/share/dict/propernames?view=co

--2022-02-14 04:18:38--  https://svnweb.freebsd.org/csrg/share/dict/words?view=co
Resolving svnweb.freebsd.org (svnweb.freebsd.org)... 96.47.72.84, 2610:1c1:1:606c::50:15
Connecting to svnweb.freebsd.org (svnweb.freebsd.org)|96.47.72.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209785 (205K) [text/plain]
Saving to: ‘words?view=co’

words?view=co       100%[===================>] 204.87K   510KB/s    in 0.4s    

2022-02-14 04:18:39 (510 KB/s) - ‘words?view=co’ saved [209785/209785]

--2022-02-14 04:18:39--  https://svnweb.freebsd.org/csrg/share/dict/connectives?view=co
Resolving svnweb.freebsd.org (svnweb.freebsd.org)... 96.47.72.84, 2610:1c1:1:606c::50:15
Connecting to svnweb.freebsd.org (svnweb.freebsd.org)|96.47.72.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706 [text/plain]
Saving to: ‘connectives?view=co’

connectives?view=co 100%[===================>]     706  --.-KB/s    in 0s      

2022-02-14 04:18:39 (128 MB/s) -

In [ ]:
# Rename dataset files
!cp words?view=co words.txt
!rm words?view=co
!cp connectives?view=co connectives.txt
!rm connectives?view=co
!cp propernames?view=co propernames.txt
!rm propernames?view=co

In [ ]:
# List of files containing datasets
file_list = { "words.txt" , "connectives.txt" , "propernames.txt" }

##Parse and Clean Datasets

###Read Datasets

In [ ]:
# Vocabulary of laguage
V = set()
# Alphabet of langauge
A = set()

In [ ]:
# Read datasets and generate vocabulary V
for dataset in file_list:
    with open ( dataset, 'r' ) as f:
        lines      = f.read ()
        lines_list = lines.split ( "\n" )
        for w in lines_list:
              V.add ( w.strip().upper() )

# Generate alphabet A from vocabulary V
for w in V:
    A.update ( set ( w ) )

##Function Definitions

##Auxilliary

In [ ]:
class S:
    """
    Represents the game state S.

    ...
    Attributes
    ----------
    _S : dict
        Dictionary representing the state of the game.

    Methods
    -------
    get_S ():
        Return the game state S.
    update ( g, e ):
        Update the game state E with the guess g and positions e.
    """
    def __init__ ( self, n ):
        """
        Inititizlize the game state S with all relevant instance attributes.

        Parameters
        ----------
        n : int
            The number of symbols n in the hidden word h_n.
        
        Returns
        -------
        None
        """
        self._S = dict ()
        for i in range ( n ):
            self._S[i] = set()
    def get_S ( self ):
        """
        Provide access to the game state S.

        Parameters
        ----------
        None
        
        Returns
        -------
        _S : dict
            Dictionary representing the state of the game.
        """
        return self._S
    def update ( self, g, e ):
        """
        Update the state of the game depending on Player alpha's evaluation of
        Player beta's guess.

        Parameters
        ----------
        g : str
            Guess g made by Player beta.
        e : set
            Set of positions i in the hidden word h_n that contain the guess g.
        
        Returns
        -------
        None
        """
        for i in self._S.keys():
            if i in e:
                self._S[i] = g
            else:
                if type ( self._S[i] ) == set:
                    self._S[i].add ( g )

class Li:
    """
    The discrete random variable representing the symbol in the i'th position of
    the hidden word h_n.

    ...
    Attributes
    ----------
    _u : set
        The positions i of h_n that have not yet been predicted by Player beta.

    Methods
    -------
    gen_Li ( a_n ):
        Generates a structure necessary for computing the probability that L_i 
        that has not yet been correctly guessed by Player beta is a_n given the 
        state of the game.
    get_u ():
        Provide access to positions of h_n that have yet been guessed u.
    get_size ():
        The number of positions that in h_n that have not yet been predicted by 
        Player beta.
    update ():
        Update u base on Player alpha's evaluation e.
    """
    def __init__ ( self, n ):
        """
        Initialize the discrete random variable with all relevant attributes.

        Parameters
        ----------
        n : int
            The number of symbols n in the hidden word h_n.
        
        Returns
        -------
        None
        """
        self._u = set ( )
        for i in range ( n ):
            self._u.add(i)
    def gen_Li ( self, a_n ):
        """
        Generates a structure used for computing the probability that the 
        L_i that has not yet been correctly guessed by Player beta assumes a 
        specific symbol given the state of the game.

        Parameters
        ----------
        a_n : str
            A symbol from the filtered vocabulary A_n.
        
        Returns
        -------
        L_i : dict
            The aforementioned structure necessary to compute the probability 
            that L_i that has not yet been correctly guessed by Player beta 
            is a_n given the state of the game.
        """
        L_i = dict()
        for i in self._u:
            L_i[i] = a_n
        return L_i
    def get_u ( self ):
        """
        Provide access to positions of h_n that have yet been guessed u.

        Parameters
        ----------
        None
        
        Returns
        -------
        _u : set
            The positions i of h_n that have not yet been predicted by Player 
            beta.
        """
        return self._u
    def get_size ( self ):
        """
        The number of positions that in h_n that have not yet been predicted by 
        Player beta.

        Parameters
        ----------
        None
        
        Returns
        -------
        |u| : int
            The number of positions in h_n that have not yet been guessed.
        """
        return len ( self._u )
    def update ( self, g, e ):
        """
        Update u base on Player alpha's evaluation e.

        Parameters
        ----------
        g : str
            Guess g made by Player beta.
        e : set
            Set of positions i in the hidden word h_n that contain the guess g.
        
        Returns
        -------
        None
        """
        for i in e:
              self._u.remove(i)

class PlayerAlpha:
    """
    Class representing Player alpha.

    ...
    Attributes
    ----------
    _V   : set
        The vocabulary V.
    _h_n : str
        The hidden word h_n.
    _n   : int
        The number of symbols n in h_n.

    Methods
    -------
    h_n ():
        Pick the hidden word h_n.
    evaluate ( g ):
        Evaluate Player beta's guess.
    """
    def __init__ ( self, V ):
        """
        Inititizlize Player alpha with all relevant instance attributes.

        Parameters
        ----------
        V : set
            The vocabulary V.
        
        Returns
        -------
        None
        """
        self._V   = V
        self._h_n = None
        self._n   = None
    def h_n ( self ):
        """
        Randomly choose a word from the vocabulary V.

        Parameters
        ----------
        None
        
        Returns
        -------
        self._n : int
            The number of symbols n in the hidden word.
        """
        self._h_n = choice ( list ( V ) )
        self._n   = len ( self._h_n )
        return self._n
    def evaluate ( self, g ):
        """
        Evaluate Player beta's guess g against the hidden word h_n.

        Parameters
        ----------
        g : str
            The symbol guessed by Player beta.

        Returns
        -------
        g, e : tuple
            g is the guess made by Player beta and e is the set of positions in
            h_n that contain the symbol g.
        """
        e = set()
        for i in range ( self._n ):
            if self._h_n[i] == g:
                e.add(i)
        return g, e

class PlayerBeta:
    """
    Class representing Player beta.

    ...
    Attributes
    ----------
    _V   : set
        The vocabulary V.
    _V_n : set
        The filtered vocabulary V_n.
    _A   : set
        The alphabet A.
    _A_n : set
        The filtered alphabet A_n.

    Methods
    -------
    filter ( n ):
        Filter the vocabulary V tp V_n and alphabet A to A_n given the 
        length of the hidden word h_n.
    predict ( Li, S ):
        Predict a symbol from the alphabet A given the game state S.
    """
    def __init__ ( self, V, A ):
        """
        Inititizlize Player beta with all relevant instance attributes.

        Parameters
        ----------
        V : set
            The vocabulary V.
        A : set
            The alphabet A.
        
        Returns
        -------
        None
        """
        self._V   = V
        self._V_n = None
        self._A   = A
        self._A_n = None
    def filter ( self, n ):
        """
        Filter the vocabulary V and alphabet A based on the number of symbols n
        in the hidden word h_n.

        Parameters
        ----------
        n : int
            The number of words n in the hidden word h_n.

        Returns
        -------
        None
        """
        self._V_n = set ()
        for w in self._V:
            if len ( w ) == n:
                self._V_n.add(w)
        self._A_n = set ()
        for w_n in self._V_n:
            self._A_n.update ( set ( w_n ) )
    def predict ( self, Li, S ):
        """
        Predict a symbol in the hidden word h_n given the game state S and the 
        number of open positions Li. This function should be overloaded by all
        child classes to implement their own prediction strategies.

        Parameters
        ----------
        Li : Li
            Open positions Li.
        S  : S
            Game state S.
        
        Returns
        -------
        NULL : str
            NULL terminator character.
        """
        return "\0"

class NaiveBeta ( PlayerBeta ):
    """
    Class representing Player beta utilizing the Naive strategy. Inherits from
    the PlayerBeta class.

    ...
    Attributes
    ----------
    _V   : set
        The vocabulary V.
    _V_n : set
        The filtered vocabulary V_n.
    _A   : set
        The alphabet A.
    _A_n : set
        The filtered alphabet A_n.

    Methods
    -------
    predict ( Li, S ):
        Predict a symbol from the alphabet usign the naive strategy given the 
        game state S.
    """
    def __init__ ( self, V, A ):
        """
        Inititizlize naive Player beta with all relevant instance attributes.

        Parameters
        ----------
        V : set
            The vocabulary V.
        A : set
            The alphabet A.
        
        Returns
        -------
        None
        """
        super ().__init__ ( V, A )
    def predict ( self, Li, E ):
        """
        Predict a symbol in the hidden word h_n given the game state S and the 
        open positions Li using the naive strategy. Overloads the 
        predict ( Li, S ) method in the parent PlayerBeta class.

        Parameters
        ----------
        Li : Li
            Open positions Li.
        S  : S
            Game state S.
        
        Returns
        -------
        g : str
            Symbol chosen by the strategy.
        """
        g = choice ( list ( self._A_n ) )
        self._A_n.remove(g)
        return g

class SemiNaiveBeta ( PlayerBeta ):
    """
    Class representing Player beta utilizing the semi-Naive strategy. Inherits 
    from the PlayerBeta class.

    ...
    Attributes
    ----------
    _V   : set
        The vocabulary V.
    _V_n : set
        The filtered vocabulary V_n.
    _A   : set
        The alphabet A.
    _A_n : set
        The filtered alphabet A_n.
    _gs  : list
        The sequences of guesses to make for the semi-naive strategy.
    _j   : int
        The number of guesses made so far j.

    Methods
    -------
    filter ( n ):
        Filter the vocabulary V tp V_n and alphabet A to A_n given the length 
        of the hidden word h_n.
    predict ( Li, S ):
        Predict a symbol from the alphabet using the semi-naive strategy given 
        the game state S.
    """
    def __init__ ( self, V, A ):
        """
        Inititizlize semi-naive Player beta with all relevant instance 
        attributes.

        Parameters
        ----------
        V : set
            The vocabulary V.
        A : set
            The alphabet A.
        
        Returns
        -------
        None
        """
        super ().__init__ ( V, A )
    def filter ( self, n ):
        """
        Filter the vocabulary V and alphabet A based on the number of symbols n
        in the hidden word h_n. Additioanlly, sort the symbols in A_n in
        descending order of frequency. Overloads the filter ( n ) method in the 
        parent class PlayerBeta.

        Parameters
        ----------
        n : int
            The number of words n in the hidden word h_n.

        Returns
        -------
        None
        """
        super ().filter ( n )
        # Compute frequency of each symbol in alphabet.
        symbol_counts = defaultdict ( int )
        for word in self._V_n:
            for symbol in word:
                symbol_counts[symbol] += 1
        temp = symbol_counts
        symbol_counts = list()
        for _l, _c in temp.items ():
            symbol_counts.append (( _c, _l ))
        # Sort the alphabet in decreasing order of frequency.
        self._gs = sorted ( symbol_counts, reverse=True )
        # Discard the frequencies and store only the symbols for later use.
        temp = self._gs
        self._gs = list()
        for _c, _l in temp:
            self._gs.append(_l)
        self._j = 0
    def predict ( self, Li, E ):
        """
        Predict a symbol in the hidden word h_n given the game state S and the 
        open positions Li using the semi-naive strategy. Overloads the 
        predict ( Li, S ) method in the parent PlayerBeta class.

        Parameters
        ----------
        Li : Li
            Open positions Li.
        S  : S
            Game state S.
        
        Returns
        -------
        g : str
            Symbol chosen by the strategy.
        """
        g = self._gs[self._j]
        self._j += 1
        self._j %= len ( self._gs )
        return g

class AdvancedBeta ( PlayerBeta ):
    """
    Class representing Player beta utilizing the advanced strategy. Inherits 
    from the PlayerBeta class.

    ...
    Attributes
    ----------
    _V   : set
        The vocabulary V.
    _V_n : set
        The filtered vocabulary V_n.
    _A   : set
        The alphabet A.
    _A_n : set
        The filtered alphabet A_n.

    Methods
    -------
    predict ( Li, S ):
        Predict a symbol from the alphabet using the advanced strategy given 
        the game state S.
    """
    def __init__ ( self, V, A ):
        """
        Inititizlize advanced Player beta with all relevant instance attributes.

        Parameters
        ----------
        V : set
            The vocabulary V.
        A : set
            The alphabet A.
        
        Returns
        -------
        None
        """
        super ().__init__ ( V, A )
    def predict ( self, Li, S ):
        """
        Predict a symbol in the hidden word h_n given the game state S and the 
        open positions Li using the advanced strategy. Overloads the 
        predict ( Li, S ) method in the parent PlayerBeta class.

        Parameters
        ----------
        Li : Li
            Open positions Li.
        S  : S
            Game state S.
        
        Returns
        -------
        g : str
            Symbol chosen by the strategy.
        """
        max_val = 0
        g = None
        for a_n in self._A_n:
            # Compute and store the value of SUM_P_W_given_S_denominator since
            # it will always produce the same output for given game state S.
            # Reuse this value since recomputing this every time makes the
            # algorithm run too slowly.
            val = P_Li_given_S ( Li, a_n, S, self._V_n, 
                                 SUM_P_W_given_S_denominator( S, self._V_n, 
                                                              cached=None ) )
            # Choose symbol with highest probability.
            if val > max_val:
                max_val = val
                g = a_n
        self._A_n.remove(g)
        return g

class Hangman:
    """
    Class representing the game of Hangman.

    ...
    Attributes
    ----------
    _player_alpha : PlayerAlpha
        Represents Player alpha.
    _player_beta  : PlayerBeta
        Represents Player beta.
    _n            : int
        The number of symbols n in hidden word h_n.
    _S            : S
        The game state S.
    _Li           : Li
        The discrete random variable representing the symbol in the i'th 
        position of the hidden word h_n.
    _j            : int
        The number of guesses made so far j.

    Methods
    -------
    play ():
        Play rounds of the game until Player beta guesses the hidden word h_n 
        chosen by Player alpha.
    swap_player ( player_beta ):
        Reset the state of the game and change the strategy being used by 
        Player beta.
    """
    def __init__ ( self, player_alpha, player_beta ):
        """
        Inititizlize Hangman game with all relevant instance attributes.

        Parameters
        ----------
        player_alpha : PlayerAlpha
            Player alpha that will play the game.
        player_beta  : PlayerBeta
            Player beta that will play the game.
        
        Returns
        -------
        None
        """
        self._player_alpha = player_alpha
        self._player_beta  = player_beta
        self._n = self._player_alpha.h_n ()
        self._S = S ( self._n )
        self._Li = Li ( self._n )
        self._player_beta.filter ( self._n )
        self._j = 0
    def play ( self ):
        """
        Play the game using the rules of Hangman until Player beta guesses all
        the symbols of the hidden word h_n chosen by Player alpha.

        Parameters
        ----------
        None

        Returns
        -------
        _j : int
            The number of guesses j made by Player beta to complete the hidden 
            word h_n.
        """
        # Player beta and Player alpha guess and evaluate repeatedly until all
        # symbols of the hidden word are predicted.
        while self._Li.get_size () > 0:
            g = self._player_beta.predict ( self._Li, self._S )
            g, e = self._player_alpha.evaluate ( g )
            self._S.update ( g, e )
            self._Li.update ( g, e )
            self._j += 1
        return self._j
    def swap_player ( self, player_beta ):
        """
        Resets the game of Hangman and swaps the the Player beta with another
        one utilizing a different strategy.

        Parameters
        ----------
        player_beta : PlayerBeta
            New Player beta utilizing a different strategy that will take the
            place of _player_beta.

        Returns
        -------
        None
        """
        self._player_beta = player_beta
        self._S = S ( self._n )
        self._Li = Li ( self._n )
        self._player_beta.filter ( self._n )
        self._j = 0

###Probabilities

In [ ]:
def P_W ( w_n, V_n ):
    """
    Computes P ( W_n = w_n ).

    Parameters
    ----------
    w_n : str
        An n symbol word from the filtered vocabulary V_n.
    V_n : set
        The filtered vocabulary V_n.
        
    Returns
    -------
    P ( W_n = w_n ) : float
        The computed probability.
    """
    return 1 / len ( V_n )

def P_Li_given_W ( i, a_n, w_n ):
    """
    Computes P ( L_i = a_n | W_n = w_n ).

    Parameters
    ----------
    i   : int
        A position within the word w_n.
    a_n : str
        A symbol from the filtered vocabulary A_n.
    w_n : str
        An n symbol word from the filtered vocabulary V_n.
        
    Returns
    -------
    P ( L_i = a_n | W_n = w_n ) : bool
        The computed probability.
    """
    if type ( a_n ) == set:
        return w_n[i] not in a_n
    else:
        return w_n[i] == a_n

def P_Li_given_W_OR ( Li, a_n, w_n ):
    """
    Computes P ( Li = a_n where i is a subset of u | W_n = w_n ).

    Parameters
    ----------
    Li  : Li
        The discrete random variable representing the symbol in the i'th 
        position of the hidden word h_n.
    a_n : str
        A symbol from the filtered vocabulary A_n.
    w_n : str
        An n symbol word from the filtered vocabulary V_n.

    Returns
    -------
    P ( Li = a_n where i is a subset of u | W_n = w_n ) : bool
        The computed probability.    
    """
    for i, _a_n in Li.gen_Li ( a_n ).items ():
        if P_Li_given_W(i, _a_n, w_n):
            return True
    return False

def P_S_given_W ( S, w_n ):
    """
    Computes P ( S | W_n = w_n ).

    Parameters
    ----------
    S   : S
        The game state S.
    w_n : str
        An n symbol word from the filtered vocabulary V_n.
        
    Returns
    -------
    P ( S | W_n = w_n ) : bool
        The computed probability.
    """
    for i, _a_n in S.get_S ().items ():
        if not P_Li_given_W ( i, _a_n, w_n ):
            return False
    return True

def SUM_P_W_given_S_denominator ( S, V_n, cached=None ):
    """
    Computes SUM ( w_n' is an element of V_n, P ( S | W_n = w_n' ) 
                                              * P ( W_n = w_n' ) )

    Parameters
    ----------
    S      : S
        The game state S.
    V_n    : set
        The filtered vocabulary V_n.
    cached : bool
        Whether to use a precomputed value of SUM_P_W_given_S_denominator.
    
    Returns
    -------
        
    """
    if cached is None:
        count_sum = 0
        for w_n_prime in V_n:
            count_sum += ( P_S_given_W ( S, w_n_prime ) * \
                           P_W ( w_n_prime, V_n ) )
        return count_sum
    else:
        return cached

def P_W_given_S ( w_n, S, V_n, cached ):
    """
    Computes P ( W_n = w_n | S).

    Parameters
    ----------
    w_n    : str
        An n symbol word from the filtered vocabulary V_n.
    S      : S
        The game state S.
    V_n    : set
        The filtered vocabulary V_n.
    cached : bool
        Whether to use a precomputed value of SUM_P_W_given_S_denominator.

    Returns
    -------
        P ( W_n = w_n | S) : float
            The computed probability.
    """
    return ( P_S_given_W ( S, w_n ) * P_W ( w_n, V_n ) ) / \
             SUM_P_W_given_S_denominator ( S, V_n, cached )
            
def P_Li_given_S ( Li, a_n, S, V_n, cached ):
    """
    Computes P ( Li = a_n where i is a subset of {1, ..., n} | S )

    Parameters
    ----------
    Li     : Li
        The discrete random variable representing the symbol in the i'th 
        position of the hidden word h_n.
    a_n    : str
        A symbol from the filtered vocabulary A_n.
    S      : S
        The game state S.
    V_n    : set
        The filtered vocabulary V_n.
    cached : bool
        Whether to use a precomputed value of SUM_P_W_given_S_denominator.

    Returns
    -------
    P ( Li = a_n where i is a subset of u | S ) : float
        The computed probability.
    """
    count_sum = 0
    for w_n in V_n:
        count_sum += ( P_Li_given_W_OR ( Li, a_n, w_n ) * \
                      P_W_given_S ( w_n, S, V_n, cached ) )
    return count_sum

##Main Script

In [ ]:
if __name__ == "__main__":
    # Lists for storing relevant performance statistics for different 
    # strategies.
    word_length = list()
    naive       = list()
    semi_naive  = list()
    advanced    = list()
    for i in range ( 9 ):
        # Initialize Player alpha and Player beta with different strategies.
        alpha =  PlayerAlpha ( V )
        beta_naive = NaiveBeta ( V, A )
        beta_sami_naive = SemiNaiveBeta ( V, A )
        beta_advanced = AdvancedBeta ( V, A )

        # Play the game using naive strategy.
        game = Hangman ( alpha, beta_naive )
        # Store the number of symbols in the hidden word.
        word_length.append ( len ( alpha._h_n ) )
        # Store number of guesses.
        naive.append ( game.play () )

        game.swap_player ( beta_sami_naive )
        # Store number of guesses.
        semi_naive.append ( game.play () )

        game.swap_player ( beta_advanced )
        # Store number of guesses.
        advanced.append ( game.play () )

In [ ]:
    # Save the performance statistics in a CSV file to be exported to
    # Microsoft Excel.
    np.savetxt('raw_data.csv', 
                np.array ( [word_length, naive, semi_naive, advanced] ).T, 
                delimiter=", ", 
                fmt= "%{}d".format(max([len(str(max(word_length))), 
                                        len(str(max(naive))), 
                                        len(str(max(semi_naive))), 
                                        len(str(max(advanced)))]) + 1),
                header= "n, naive, semi-naive, advanced",
                comments= "")